In [2]:
import numpy as np # library to handle data in a vectorized manner
!pip install geopy

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files"

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 112kB 6.0MB/s eta 0:00:01
Libraries imported.


In [5]:
schdf = pd.read_csv('school.csv')
schdf = schdf.drop(columns=['_id', 'OBJECTID', 'GEO_ID', 'SCHOOL_LEVEL', 'ADDRESS_POINT_ID', 'ADDRESS_NUMBER', 'LINEAR_NAME_FULL', 'GENERAL_USE_CODE', 'CENTRELINE_ID', 'LO_NUM', 'LO_NUM_SUF', 'HI_NUM', 'HI_NUM_SUF', 'LINEAR_NAME_ID', 'X', 'Y', 'geometry'])

print('The dataframe has {} types and {} schools.'.format(
        len(schdf['SCHOOL_TYPE_DESC'].unique()),
        schdf.shape[0]
    )
)
schdf['SCHOOL_TYPE_DESC'].unique()
#schdf.groupby(['SCHOOL_TYPE_DESC']).size()
schdf.head()

The dataframe has 7 types and 1194 schools.


,NAME,SCHOOL_TYPE,BOARD_NAME,SOURCE_ADDRESS,SCHOOL_TYPE_DESC,ADDRESS_FULL,POSTAL_CODE,MUNICIPALITY,CITY,PLACE_NAME,LATITUDE,LONGITUDE
0,A PLUS ACADEMY OF ADVANCEMENT,PR,NaN,2425 EGLINTON AVE E,Private,2425 Eglinton Ave E,M1K 5G8,Scarborough,Toronto,NaN,43.732091,-79.267107
1,A R S ARMENIAN PRIVATE SCHOOL,PR,NaN,50 HALLCROWN PL,Private,50 Hallcrown Pl,M2J 1P7,North York,Toronto,NaN,43.770062,-79.323159
2,A Y J GLOBAL ACADEMY,PR,NaN,4 LANSING SQ,Private,4 Lansing Sq,M2J 5A2,North York,Toronto,NaN,43.774091,-79.325220
3,A Y JACKSON SECONDARY SCHOOL,EP,Toronto District School Board,50 FRANCINE DR,English Public,50 Francine Dr,M2H 2G6,North York,Toronto,A. Y. Jackson Secondary School,43.805270,-79.366559
4,ABACUS MONTESSORI LEARNING CENTRE,PR,NaN,4 CREDIT UNION DR,Private,4 Credit Union Dr,M4A 2N8,North York,Toronto,Latvian Canadian Centre,43.725441,-79.319013


In [6]:
lats = [43.787979, 43.686821, 43.733830, 43.723721, 43.686550, 43.659970, 45.015960, 43.655870, 43.661660, 43.719530, 43.707620, 43.723720,
       43.661660, 43.662330, 43.669600, 43.682760, 43.655450, 43.661660, 43.683490, 43.660510, 43.671560, 43.675530, 43.669600, 43.802020,
       43.719320, 43.658760, 43.631930, 43.709720, 43.656570, 43.661660,
       43.656700, 43.649190, 43.668040, 43.950370, 43.661440]

long = [-79.190964, -79.393593, -79.628680, -79.408783, -79.324640, -79.378870, -74.737000, -79.409230, -79.387730, -79.444960, -79.376390, -79.408780,
       -79.387730, -79.366460, -79.397480, -79.326390, -79.457830, -79.387730, -79.471850, -79.374300, -79.493360, -79.329030, -79.397480, -79.308600,
       -79.572090, -79.381570, -79.355290, -79.399460, -79.379340, -79.387730,
       -79.451640, -79.483460, -79.492400, -78.307740, -79.383490]

latlon = pd.DataFrame({'LATITUDE':lats, 'LONGITUDE':long})

In [13]:
sexdf = pd.read_csv('sexhealth.csv')
sexdf = sexdf.drop(columns=['_id', 'OBJECTID', 'NEIGHBOURHOOD', 'ADDRESS_POINT_ID', 'X', 'Y', 'LONGITUDE', 'LATITUDE', 'geometry'])
sexdf['LATITUDE'] = latlon['LATITUDE']
sexdf['LONGITUDE'] = latlon['LONGITUDE']

pd.set_option('display.max_colwidth', 100)
#sexdf.head()

In [12]:
x = 0
accs = []
preg = []
cons = []
test = []
birt = []
relt = []
abrt = []
sxtr = []
educ = []

for desc in zip(sexdf['DESCRIPTION_SERVICE']):
    if (pd.isnull(sexdf['ACCESSIBILITY'][x])):
        accs.append(0)
    else:
        accs.append(1)
    if "pregnancy" in sexdf['DESCRIPTION_SERVICE'][x]:
        preg.append(1)
    else:
        preg.append(0)
    if "counselling" in sexdf['DESCRIPTION_SERVICE'][x]:
        cons.append(1)
    else:
        cons.append(0)
    if "testing" in sexdf['DESCRIPTION_SERVICE'][x]:
        test.append(1)
    else:
        test.append(0)
    if "birth control" in sexdf['DESCRIPTION_SERVICE'][x]:
        birt.append(1)
    else:
        birt.append(0)
    if "relationship" in sexdf['DESCRIPTION_SERVICE'][x]:
        relt.append(1)
    else:
        relt.append(0)
    if "abortion" in sexdf['DESCRIPTION_SERVICE'][x]:
        abrt.append(1)
    else:
        abrt.append(0)
    if "sexually transmitted" in sexdf['DESCRIPTION_SERVICE'][x]:
        sxtr.append(1)
    else:
        sxtr.append(0)
    if "education" in sexdf['DESCRIPTION_SERVICE'][x]:
        educ.append(1)
    else:
        educ.append(0)
    
    x += 1

nsexdf = pd.DataFrame()
nsexdf['CENTER SERVICES PROVIDED: '] = sexdf['AGENCY_NAME']
nsexdf['ACCESSIBILITY INFO GIVEN'] = accs
nsexdf['PREGNANCY'] = preg
nsexdf['COUNSELLING'] = cons
nsexdf['TESTING'] = test
nsexdf['BIRTH CONTROL'] = birt
nsexdf['RELATIONSHIP'] = relt
nsexdf['ABORTION'] = abrt
nsexdf['SEXUALLY TRASMITTED'] = sxtr
nsexdf['EDUCATION'] = educ

nsexdf

,CENTER SERVICES PROVIDED:,ACCESSIBILITY INFO GIVEN,PREGNANCY,COUNSELLING,TESTING,BIRTH CONTROL,RELATIONSHIP,ABORTION,SEXUALLY TRASMITTED,EDUCATION
0,University of Toronto,1,1,1,1,1,0,0,1,0
1,Talk Shop Sexual Health Clinic,1,1,1,1,1,1,0,1,0
2,Seneca College - Newnham Campus - Student Health Centre,0,0,1,1,1,0,0,1,0
3,Birth Control and VD Information Centre,1,1,1,1,1,0,0,1,1
4,Birthright,1,1,1,1,0,0,0,0,0
5,Hassle Free Clinic,1,1,1,1,1,1,1,1,0
6,"University Health Network, Toronto Western Hospital",1,0,0,0,0,0,0,0,0
7,Immigrant Women's Health Centre,1,1,1,0,1,0,0,1,1
8,Women's College Hospital,1,1,0,0,0,0,0,0,1
9,Unison Health and Community Services,1,1,1,1,0,0,0,1,0


In [9]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, nm, scht, schtd, addr in zip(schdf['LATITUDE'], schdf['LONGITUDE'], schdf['NAME'], 
                                                schdf['SCHOOL_TYPE'], schdf['SCHOOL_TYPE_DESC'], schdf['ADDRESS_FULL']):
    label = '{}, {}, {}'.format(nm, schtd, addr)
    label = folium.Popup(label, parse_html=True)
    
    colorl = ""
    fcolorl = ""
    if(schtd == "Private"):
        colorl = 'blue'
        fcolorl = '#3186cc'
    elif(schtd == "English Separate" or schtd == "English Public"):
        colorl = 'green'
        fcolorl = 'lightgreen'
    elif(schtd == "French Separate" or schtd == "French Public"):
        colorl = 'gold'
        fcolorl = 'yellow'
    else:
        colorl = 'white'
        fcolorl = 'lightgray'
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colorl,
        fill=True,
        fill_color=fcolorl,
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
# add markers to map
x = 0
for lat, lng, nm, addr in zip(sexdf['LATITUDE'], sexdf['LONGITUDE'], sexdf['AGENCY_NAME'], sexdf['ORGANIZATION_ADDRESS']):
    
    colorl = ""
    fcolorl = ""
    
    label = ""
    label += nm
    label += (", " + addr)
    
    if sexdf['OFFICE_PHONE'][x] is not None:
        label += (", " + sexdf['OFFICE_PHONE'][x])
    if sexdf['EMAIL'][x] is not None:
        label += (", " + str(sexdf['EMAIL'][x]))
    if sexdf['ELIGIBILITY'][x] is not None:
        label += (", " + str(sexdf['ELIGIBILITY'][x]))
    
    if(pd.isnull(sexdf['ACCESSIBILITY'][x])):
        colorl = 'darkred'
        fcolorl = 'darkred'
    else:
        label += (", " + sexdf['ACCESSIBILITY'][x])
        colorl = 'red'
        fcolorl = 'darkred'
        
    #print(label)
    label = folium.Popup(label, parse_html=True)
    
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        popup=label,
        color=colorl,
        radius=8,
        #fill=True,
        fill_color=fcolorl,
        fill_opacity=1
        #parse_html=False
    ).add_to(map_toronto)  
    
    x += 1
    
map_toronto